In [168]:
%load_ext autoreload
%autoreload
%matplotlib inline

import numpy as np
import pandas as pd
import gensim
from utils import *

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [212]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
market_master = load_prices("C:\\Users\\Vojta-Acer\Desktop\\Diplomka\\dataMarket\\GOOG.csv")
tweets = load_tweets("C:\\Users\\Vojta-Acer\\Desktop\Diplomka\\dataProcessed\\tweetsGOOG.csv")

In [23]:
#tweets_hour = aggregate_tweets(tweets, 'hour', 'tokens')
tweets_5min = aggregate_tweets(tweets, '5min', 'tokens')
#tweets_min = aggregate_tweets(tweets, 'min', 'tokens')
#tweets_none = aggregate_tweets(tweets, 'none', 'tokens')

In [221]:
tweet_count = tweets.groupby('date').size()
tweet_count.index = pd.to_datetime(tweet_count.index) 

market = market_master['Close'].between_time('16:00', '16:00')
market.index = market.index.date

df = pd.DataFrame(tweet_count.rename('Tweet Count')).join(market, how = 'inner')
df['RET'] = df['Close'].pct_change()
df['VOL'] = df['RET']*df['RET']


In [222]:

df['Count lag 1'] = df['Tweet Count'].shift()
df['Count lag 2'] = df['Tweet Count'].shift(2)
df['Count lag 3'] = df['Tweet Count'].shift(3)
df['Count lag 4'] = df['Tweet Count'].shift(4)
df['Count lag 5'] = df['Tweet Count'].shift(5)

df = df.dropna()

In [251]:
features = df.iloc[:,[0,4,5,6,7,8]]
features = df['Count lag 4']
features = np.log(features)
labels = df['VOL']

mod = sm.OLS(labels, features)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    VOL   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     33.39
Date:                Mon, 23 Apr 2018   Prob (F-statistic):           3.80e-08
Time:                        12:24:10   Log-Likelihood:                 1039.2
No. Observations:                 162   AIC:                            -2076.
Df Residuals:                     161   BIC:                            -2073.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Count lag 4  2.667e-05   4.62e-06      5.778      